# 파이프라인 테스트

## 1. 준비

In [ ]:
import requests
import urllib.parse
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread
import pytz
from datetime import datetime, timedelta
import requests
import FinanceDataReader as fdr
import pandas as pd
import tqdm
import time
from melo.api import TTS
from api_wrapper import ApiWrapper
from dotenv import load_dotenv
from os import getenv

load_dotenv("./")


BASE_URL = getenv("BASE_URL")

AUTH = {
    "userId": getenv("AUTH_USER_ID"),
    "password": getenv("AUTH_PASSWORD")
}

result = requests.post(BASE_URL + "auth/token", json=AUTH)

if result.status_code != 200:
    raise RuntimeError("login error!")

result = json.loads(result.text)
TOKEN = result["data"][0]["accessToken"]
print(TOKEN)


tickers = {
    "AAPL": "애플",
    "ABNB": "에어비앤비",
    "ADBE": "어도비",
    "ADI": "아날로그 디바이스",
    "ADP": "오토매틱 데이터 프로세싱",
    "ADSK": "오토데스크",
    "AEP": "아메리칸 일렉트릭 파워",
    "AMAT": "어플라이드 머티어리얼즈",
    "AMD": "어드밴스트 마이크로 디바이시스 (AMD)",
    "AMGN": "암젠",
    "AMZN": "아마존",
    "ANSS": "앤시스",
    "APP": "애플로빈",
    "ARM": "암홀딩스",
    "ASML": "ASML",
    "AVGO": "브로드컴",
    "AXON": "액손 엔터프라이즈",
    "AZN": "아스트라제네카",
    "BIIB": "바이오젠",
    "BKNG": "부킹 홀딩스",
    "BKR": "베이커 휴즈",
    "CCEP": "코카콜라 유로퍼시픽 파트너스",
    "CDNS": "케이던스 디자인 시스템즈",
    "CDW": "CDW",
    "CEG": "콘스텔레이션 에너지",
    "CHTR": "차터 커뮤니케이션즈",
    "CMCSA": "컴캐스트",
    "COST": "코스트코",
    "CPRT": "코파트",
    "CRWD": "크라우드스트라이크",
    "CSCO": "시스코",
    "CSGP": "코스타 그룹",
    "CSX": "CSX",
    "CTAS": "신타스",
    "CTSH": "코그니전트",
    "DASH": "도어대시",
    "DDOG": "데이터독",
    "DXCM": "덱스컴",
    "EA": "일렉트로닉 아츠",
    "EXC": "엑셀론",
    "FANG": "다이아몬드백 에너지",
    "FAST": "패스널",
    "FTNT": "포티넷",
    "GEHC": "GE 헬스케어",
    "GFS": "글로벌파운드리즈",
    "GILD": "길리어드 사이언스",
    "GOOG": "알파벳 C",
    "GOOGL": "알파벳 A",
    "HON": "허니웰",
    "IDXX": "아이덱스",
    "INTC": "인텔",
    "INTU": "인튜잇",
    "ISRG": "인튜이티브 서지컬",
    "KDP": "큐리그 닥터페퍼",
    "KHC": "크래프트 하인즈",
    "KLAC": "KLA",
    "LIN": "린데",
    "LRCX": "램리서치",
    "LULU": "룰루레몬",
    "MAR": "메리어트",
    "MCHP": "마이크로칩 테크놀로지",
    "MDB": "몽고DB",
    "MDLZ": "몬덜리즈",
    "MELI": "메르카도리브레",
    "META": "메타",
    "MNST": "몬스터 베버리지",
    "MRVL": "마벨 테크놀로지",
    "MSFT": "마이크로소프트",
    "MSTR": "마이크로스트래티지",
    "MU": "마이크론",
    "NFLX": "넷플릭스",
    "NVDA": "엔비디아",
    "NXPI": "NXP",
    "ODFL": "올드 도미니언",
    "ON": "온 세미컨덕터",
    "ORLY": "오라일리 오토모티브",
    "PANW": "팔로알토 네트웍스",
    "PAYX": "페이첵스",
    "PCAR": "팩카",
    "PDD": "핀둬둬",
    "PEP": "펩시코",
    "PLTR": "팔란티어",
    "PYPL": "페이팔",
    "QCOM": "퀄컴",
    "REGN": "리제네론",
    "ROP": "로퍼 테크놀로지스",
    "ROST": "로스 스토어즈",
    "SBUX": "스타벅스",
    "SNPS": "시놉시스",
    "TEAM": "아틀라시안",
    "TMUS": "T-모바일",
    "TSLA": "테슬라",
    "TTD": "더 트레이드 데스크",
    "TTWO": "테이크투 인터랙티브",
    "TXN": "텍사스 인스트루먼트",
    "VRSK": "베리스크",
    "VRTX": "버텍스 파마슈티컬",
    "WBD": "워너 브라더스 디스커버리",
    "WDAY": "워크데이",
    "XEL": "엑셀 에너지",
    "ZS": "지스케일러",
}

## 2. 데이터 준비

In [ ]:
api = ApiWrapper()

news_list = api.get_news_data()

model_name = "LGAI-EXAONE/EXAONE-Deep-7.8B"
streaming = False    # choose the streaming option


## 3. 업로드

In [ ]:
for ticker in tqdm.tqdm(tickers.keys()):
    target = news_list[ticker]
    name = tickers[ticker]
    data = []
    for i in target:
        data.append(i["title"].replace(":", "")  + " : " + i["description"].replace(":", ""))
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)


    prompt = \
    f"""
    당신은 탁월한 증권사 애널리스트입니다.
    당신은 지금 {name} 종목의 분석 보고서를 발간해야 합니다.

    [요구 사항]
    {name} 기업의 주요 헤드라인을 검토하고, 분석 보고서에 들어갈 시황, 주요 이슈를 2000자로 구성된 {name} 기업 분석 보고서를 만들어 주세요.
    개인 투자자 입장에서 주의하거나 생각해봐야 할 시사점들이 있다면 분석 보고서에 반영해 주세요.
    주요 헤드라인의 모든 내용이 {name} 기업과 연관되어 있지 않을 수도 있습니다. 이 경우 관련 내용을 담은 헤드라인만 보고서에 반영해 주세요.

    [참고 사항]
    주요 헤드라인은 한 줄의 '[기사 제목]: [description]' 으로 구성되어 있습니다.

    [이 시각 {name} 기업의 주요 헤드라인]
    {"\n".join(data)}

    """

    messages = [
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
    )
    result = tokenizer.decode(output[0]).split("</thought>")[1].replace("[|endofturn|]", "")
    prompt = \
    f"""
    당신은 탁월한 경제 연구원입니다.
    당신은 지금 {name} 종목의 분석 보고서를 발간해야 합니다.

    [요구 사항]
    다음 [초안]을 보고, 세 문단으로 구성된 보고서 최종본을 만들어 주세요.

    [제약 사항]
    각각의 문단은 연속되고 완결된 문장으로 구성되어야 합니다.
    별도의 제목이나 메타 정보는 생략 하십시오.
    단락 구분 없이, 문장을 연속된 3개의 문단으로 완성해 주세요.
    문장 간 연결은 자연스럽고 매끄러워야 합니다. 필요하다면 문단을 추가할 수 있습니다.

    [초안]
    {result}
    """

    messages = [
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
    )
    result = tokenizer.decode(output[0]).split("</thought>")[1].replace("[|endofturn|]", "")
    print(result)


    kst = pytz.timezone('Asia/Seoul')

    data = {
    "ticker": ticker,
    "uploadDate": datetime.now(kst).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z",
    "summaryText": result
    }

    res = requests.post(
        BASE_URL + "upload/summary",
        json=data,
        headers={"Authorization": "Bearer " + TOKEN}
    )

    if res.status_code != 200:
        print(res.status_code)
        print(res.text)
        raise RuntimeError("news data upload error!")

    # Speed is adjustable
    speed = 1.15
    device = 'cpu' # or cuda:0

    text = result
    model = TTS(language='KR', device=device)
    speaker_ids = model.hps.data.spk2id

    output_path = ticker + '.wav'
    model.tts_to_file(text, speaker_ids['KR'], output_path, speed=speed)


## 4. 주가, 환율, 유튜브 링크

In [ ]:
def calculate_buy_index(df: pd.DataFrame) -> float:
    close = df['Close']
    high = df['High']
    low = df['Low']

    # Bollinger Band
    sma = close.rolling(20).mean()
    std = close.rolling(20).std()
    upper = sma + 2 * std
    lower = sma - 2 * std
    boll = (
        "Buy" if close.iloc[-1] <= lower.iloc[-1]
        else "Sell" if close.iloc[-1] >= upper.iloc[-1] and close.iloc[-1] < sma.iloc[-1]
        else "Hold"
    )

    # MACD
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    macd_s = (
        "Buy" if macd.iloc[-1] < 0 and macd.iloc[-2] < signal.iloc[-2] and macd.iloc[-1] > signal.iloc[-1]
        else "Sell" if macd.iloc[-1] > 0 and macd.iloc[-2] > signal.iloc[-2] and macd.iloc[-1] < signal.iloc[-1]
        else "Hold"
    )

    # RSI
    delta = close.diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    rsi_s = (
        "Buy" if rsi.iloc[-2] < 30 and rsi.iloc[-1] >= 30
        else "Sell" if rsi.iloc[-2] > 70 and rsi.iloc[-1] <= 70
        else "Hold"
    )

    # Stochastic
    lowest = low.rolling(14).min()
    highest = high.rolling(14).max()
    k = 100 * (close - lowest) / (highest - lowest)
    d = k.rolling(3).mean()
    sto = (
        "Buy" if k.iloc[-1] <= 20 and k.iloc[-2] < d.iloc[-2] and k.iloc[-1] > d.iloc[-1]
        else "Sell" if k.iloc[-1] >= 80 and k.iloc[-2] > d.iloc[-2] and k.iloc[-1] < d.iloc[-1]
        else "Hold"
    )

    # 시그널 → 값
    def signal_to_value(signal: str) -> float:
        return {"Buy": 1.0, "Hold": 0.5, "Sell": 0.0}[signal]

    # 통합
    weights = {"bollinger": 0.2, "macd": 0.3, "rsi": 0.2, "stochastic": 0.4}
    score = (
        signal_to_value(boll) * weights["bollinger"] +
        signal_to_value(macd_s) * weights["macd"] +
        signal_to_value(rsi_s) * weights["rsi"] +
        signal_to_value(sto) * weights["stochastic"]
    )
    print(score)
    return round(score, 4)


for ticker in tqdm.tqdm(tickers.keys()):
    df: pd.DataFrame = fdr.DataReader(ticker, (datetime.now() - timedelta(days=365)).strftime("%Y%m%d"), datetime.now().strftime("%Y%m%d"))
    high52week, low52week = max(df.Close), min(df.Close)
    currentPrice = df.Close.iloc[-1]

    stock_data = {
        datetime.fromtimestamp(int(ts) / 1000).strftime('%Y-%m-%d'): val
        for ts, val in json.loads(df.T.to_json()).items()
    }

    df = fdr.DataReader(ticker, start=(datetime.now() - timedelta(days=365)).strftime("%Y%m%d"))
    # print(df)

    buy_index = calculate_buy_index(df) or 0.5


    data = {
        "Authentication": TOKEN,
        "ticker": ticker,
        "companyName": "tesla",
        "currentPrice": currentPrice,
        "previousClosePrice": currentPrice,
        "high52week": high52week,
        "low52week": low52week,
        "buyIndex": buy_index,
        "prices": stock_data,
    }


    result = requests.post(BASE_URL + "raw", json=data)

    if result.status_code != 200:
        raise RuntimeError("raw stock data upload error!")


queries = urllib.parse.urlencode({
    "authkey": getenv("EXCHANGE_RATE_API_KEY"),
    # "searchdate": None, # 아무것도 없으면 현재일 입니다
    "data": "AP01" # AP01은 환율 AP02는 대출 금리 AP03은 국제 금리 입니다
})
# Ih8j11Qqm7mng1H1c3A1n9hQRGuRHExN

retry = 3
while retry:
    try:
        response = requests.get(
            url="https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?" + queries,
            verify=False
        )

        result = None
        if response.status_code == 200:
            result = json.loads(response.text)
            for i in result:
                if "cur_nm" in i.keys() and i["cur_nm"] == '미국 달러':
                    result = i
                    break

        data = {}
        data["buyRate"] = float(result["ttb"].replace(",", ""))
        data["sellRate"] = float(result["tts"].replace(",", ""))
        data["date"] = datetime.now().strftime("%Y-%m-%d")
        data["dealBaseRate"] = float(result["deal_bas_r"].replace(",", ""))
        data["currency"] = "USD"
        break
    except:
        retry -= 1
        continue

result = requests.post(BASE_URL + "exchange/save", json=data)

if result.status_code != 200:
    raise RuntimeError("exchange rate data upload error!")


In [ ]:
for ticker in tickers.keys():
    url = f"https://www.googleapis.com/youtube/v3/search?q={tickers[ticker]}&part=snippet&key={getenv('YOUTUBE_API_KEY')}"
    result = requests.get(url).json()

    # for e in result["items"]:
    e = result["items"][0]

    vid = e["id"]["videoId"]
    title = e["snippet"]["title"]
    link = e["snippet"]["thumbnails"]["high"]["url"]

    data = {
        "ticker": ticker,
        "title": title,
        "thumbnailUrl": link,
        "videoId": vid,
        "publishedAt": e["snippet"]["publishedAt"],
    }

    result = requests.post(BASE_URL + "upload/youtube", json=data, headers={"Authorization": TOKEN})

    if result.status_code != 200:
        raise RuntimeError("youtube data upload error!")
    
    time.sleep(0.1)
